In [43]:
#needed to make web requests
# REST APIs
import requests

#store the data we get as a dataframe
import pandas as pd

#convert the response as a strcuctured json
import json

#mathematical operations on lists
import numpy as np

#parse the datetimes we get from NOAA
from datetime import datetime

import pickle


# Enter Boulder coordinates

In [2]:
# Enter coordinates 
latitude=40.014984
longitude=-105.270546

In [4]:
# I could request more than one
resp = requests.get('https://api.weather.gov/points/{},{}'.format(latitude,longitude))




In [5]:
print(type(resp))

<class 'requests.models.Response'>


In [6]:
d = resp.json()

In [7]:
d

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   's': 'https://schema.org/',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#',
   'geometry': {'@id': 's:GeoCoordinates', '@type': 'geo:wktLiteral'},
   'city': 's:addressLocality',
   'state': 's:addressRegion',
   'distance': {'@id': 's:Distance', '@type': 's:QuantitativeValue'},
   'bearing': {'@type': 's:QuantitativeValue'},
   'value': {'@id': 's:value'},
   'unitCode': {'@id': 's:unitCode', '@type': '@id'},
   'forecastOffice': {'@type': '@id'},
   'forecastGridData': {'@type': '@id'},
   'publicZone': {'@type': '@id'},
   'county': {'@type': '@id'}}],
 'id': 'https://api.weather.gov/points/40.015,-105.2705',
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-105.2705, 40.015]},
 'properties': {'@id': 'https://api.weather.gov/p

In [8]:
d.keys()

dict_keys(['@context', 'id', 'type', 'geometry', 'properties'])

In [9]:
d['properties']['forecast']

'https://api.weather.gov/gridpoints/BOU/53,74/forecast'

# Query the forecast

In [10]:
resp_forecast = requests.get((d['properties']['forecast']))

In [11]:
d_forecast = resp_forecast.json()

In [12]:
d_forecast

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-105.2923958, 40.031166],
    [-105.29021449999999, 40.0092019],
    [-105.26157919999999, 40.0108675],
    [-105.26375449999999, 40.032831800000004],
    [-105.2923958, 40.031166]]]},
 'properties': {'updated': '2021-05-04T01:38:33+00:00',
  'units': 'us',
  'forecastGenerator': 'BaselineForecastGenerator',
  'generatedAt': '2021-05-04T02:45:09+00:00',
  'updateTime': '2021-05-04T01:38:33+00:00',
  'validTimes': '2021-05-03T19:00:00+00:00/P7DT11H',
  'elevation': {'value': 1645.92, 'unitCode': 'unit:m'},
  'periods': [{'number': 1,
    'name': 'Tonight',
    'startTime': '2021-05-03T20:00:00-06:00',
    'endTime': '2021-05-04T06

# Query the hourly forecast

In [14]:
resp_hourly = requests.get((d['properties']['forecastHourly']))

In [19]:
type(resp_hourly)

requests.models.Response

In [17]:
d_hourly = resp_hourly.json()

In [18]:
d_hourly

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-105.2923958, 40.031166],
    [-105.29021449999999, 40.0092019],
    [-105.26157919999999, 40.0108675],
    [-105.26375449999999, 40.032831800000004],
    [-105.2923958, 40.031166]]]},
 'properties': {'updated': '2021-05-04T01:38:33+00:00',
  'units': 'us',
  'forecastGenerator': 'HourlyForecastGenerator',
  'generatedAt': '2021-05-04T02:47:54+00:00',
  'updateTime': '2021-05-04T01:38:33+00:00',
  'validTimes': '2021-05-03T19:00:00+00:00/P7DT11H',
  'elevation': {'value': 1645.92, 'unitCode': 'unit:m'},
  'periods': [{'number': 1,
    'name': '',
    'startTime': '2021-05-03T20:00:00-06:00',
    'endTime': '2021-05-03T21:00:00-06

In [21]:
d_hourly.keys()

dict_keys(['@context', 'type', 'geometry', 'properties'])

In [22]:
d_hourly_subset = d_hourly['properties']

In [23]:
d_hourly_subset.keys()

dict_keys(['updated', 'units', 'forecastGenerator', 'generatedAt', 'updateTime', 'validTimes', 'elevation', 'periods'])

In [25]:
updated_time = d_hourly_subset['updateTime']

In [30]:
forecast_per_hour_rain = d_hourly_subset['periods']

This is a list of dictionaries - parse into a panda structure. Every dictionary corresponds to a measured period and in this case will be one hour. 

In [31]:
forecast_per_hour_rain

[{'number': 1,
  'name': '',
  'startTime': '2021-05-03T20:00:00-06:00',
  'endTime': '2021-05-03T21:00:00-06:00',
  'isDaytime': False,
  'temperature': 39,
  'temperatureUnit': 'F',
  'temperatureTrend': None,
  'windSpeed': '2 mph',
  'windDirection': 'NNW',
  'icon': 'https://api.weather.gov/icons/land/night/rain_showers,60?size=small',
  'shortForecast': 'Rain Showers Likely',
  'detailedForecast': ''},
 {'number': 2,
  'name': '',
  'startTime': '2021-05-03T21:00:00-06:00',
  'endTime': '2021-05-03T22:00:00-06:00',
  'isDaytime': False,
  'temperature': 40,
  'temperatureUnit': 'F',
  'temperatureTrend': None,
  'windSpeed': '5 mph',
  'windDirection': 'N',
  'icon': 'https://api.weather.gov/icons/land/night/rain_showers,60?size=small',
  'shortForecast': 'Rain Showers Likely',
  'detailedForecast': ''},
 {'number': 3,
  'name': '',
  'startTime': '2021-05-03T22:00:00-06:00',
  'endTime': '2021-05-03T23:00:00-06:00',
  'isDaytime': False,
  'temperature': 40,
  'temperatureUnit':

In [37]:
#To save into csv file pickle so I have different forecast to play with in the future. This will be rainy.

In [44]:
#dictionary of drives into a pickle object under the DATA folder 
with open('forecast_rain_0503.pickle', 'wb') as handle:
    pickle.dump(forecast_per_hour_rain, handle, protocol = pickle.HIGHEST_PROTOCOL)
    
#with open('/home/cdsw/DATA/dict_drivessuccess_0_4800_n30_v6b.pickle', 'rb') as handle:
#    dict_drives_train_success = pickle.load(handle)

# Pandas manipulation 

In [38]:
#parse first element in the list
type(forecast_per_hour_rain[0])

dict

In [36]:
df_per_hour= pd.DataFrame(forecast_per_hour_rain)

In [39]:
df_per_hour

,detailedForecast,endTime,icon,isDaytime,name,number,shortForecast,startTime,temperature,temperatureTrend,temperatureUnit,windDirection,windSpeed
0,,2021-05-03T21:00:00-06:00,https://api.weather.gov/icons/land/night/rain_...,False,,1,Rain Showers Likely,2021-05-03T20:00:00-06:00,39,None,F,NNW,2 mph
1,,2021-05-03T22:00:00-06:00,https://api.weather.gov/icons/land/night/rain_...,False,,2,Rain Showers Likely,2021-05-03T21:00:00-06:00,40,None,F,N,5 mph
2,,2021-05-03T23:00:00-06:00,https://api.weather.gov/icons/land/night/rain_...,False,,3,Rain Showers Likely,2021-05-03T22:00:00-06:00,40,None,F,N,6 mph
3,,2021-05-04T00:00:00-06:00,https://api.weather.gov/icons/land/night/rain_...,False,,4,Rain Showers Likely,2021-05-03T23:00:00-06:00,38,None,F,NW,3 mph
4,,2021-05-04T01:00:00-06:00,https://api.weather.gov/icons/land/night/bkn?s...,False,,5,Mostly Cloudy,2021-05-04T00:00:00-06:00,38,None,F,W,2 mph
5,,2021-05-04T02:00:00-06:00,https://api.weather.gov/icons/land/night/bkn?s...,False,,6,Mostly Cloudy,2021-05-04T01:00:00-06:00,38,None,F,WNW,2 mph
6,,2021-05-04T03:00:00-06:00,https://api.weather.gov/icons/land/night/bkn?s...,False,,7,Mostly Cloudy,2021-05-04T02:00:00-06:00,38,None,F,WNW,2 mph
7,,2021-05-04T04:00:00-06:00,https://api.weather.gov/icons/land/night/bkn?s...,False,,8,Mostly Cloudy,2021-05-04T03:00:00-06:00,37,None,F,NW,3 mph
8,,2021-05-04T05:00:00-06:00,https://api.weather.gov/icons/land/night/bkn?s...,False,,9,Mostly Cloudy,2021-05-04T04:00:00-06:00,36,None,F,WNW,5 mph
9,,2021-05-04T06:00:00-06:00,https://api.weather.gov/icons/land/night/bkn?s...,False,,10,Mostly Cloudy,2021-05-04T05:00:00-06:00,35,None,F,W,5 mph


In [41]:
# Sleect the interesting columns

df_per_hour.columns

Index(['detailedForecast', 'endTime', 'icon', 'isDaytime', 'name', 'number',
       'shortForecast', 'startTime', 'temperature', 'temperatureTrend',
       'temperatureUnit', 'windDirection', 'windSpeed'],
      dtype='object')